In [3]:
import cloudinary
import cloudinary.uploader
from cloudinary.utils import cloudinary_url

# Configuration       
cloudinary.config( 
    cloud_name = "dzr5vt5ut", 
    api_key = "829673595861745", 
    api_secret = "9rWMNm7vKvVMBEPzqiD2E8-YALE", # Click 'View Credentials' below to copy your API secret
    secure=True
)

# Upload an image
upload_result = cloudinary.uploader.upload("D:\\Work and Assignments\\Python\\Python Projects\\Google Vision API\\dp.jpg")
print(upload_result["secure_url"])


https://res.cloudinary.com/dzr5vt5ut/image/upload/v1718101715/f6n79fx39xol0fwovji5.jpg


In [5]:
import pandas as pd
c = pd.read_csv("D:\\Work and Assignments\\Python\\Product_DB.csv")
c

,product_name,product_url
0,Puma Unisex-Adult Smashic Casual Shoe,https://m.media-amazon.com/images/I/51MiYR1te6...
1,NIKE Womens W Revolution 7 Running Shoe,https://m.media-amazon.com/images/I/71cy1dbMh5...
2,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
3,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
4,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...
5,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...
6,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...
7,Bata Mens MaxUniform Dress Shoe,https://m.media-amazon.com/images/I/710+f7XX2F...
8,SPARX Mens Sx0675g Walking Shoe,https://m.media-amazon.com/images/I/71u2XOFXAI...
9,Campus Stack Sneakers for Men,https://m.media-amazon.com/images/I/61U0BBQYhr...
